# Import and Reading of csv



In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score
from sklearn.preprocessing import label_binarize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.base import clone
from sklearn.preprocessing import LabelEncoder
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

In [ ]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [ ]:
mosq_csv = pd.read_csv('/content/drive/MyDrive/CMP466 - Machine Learning/Project/MTM_DISCRIMINATING_CONCENTRATION_BIOASSAY_MOD.csv')

# Preprocessing

In [ ]:
mosq_csv.nunique()

ID                       21074
COUNTRY_NAME                87
ISO2                        86
ADMIN1                     721
ADMIN2                    1913
SITE_NAME                 3782
SITE_CODE                 3793
LATITUDE                  3624
LONGITUDE                 3635
ASSAY_TYPE                   1
TEST_TYPE                    3
INSECTICIDE_CLASS            6
INSECTICIDE_TYPE            18
INSECTICIDE_CONC            18
INSECTICIDE_INTENSITY        1
YEAR_START                  44
SPECIES                     92
STAGE_ORIGIN                 5
MOSQUITO_NUMBER            431
TEST_TIME                    0
MORTALITY_ADJUSTED        1594
RESISTANCE_STATUS            4
DATA_SOURCE                264
CITATION                   660
CITATION_URL               447
DATA_CURATOR                 3
dtype: int64

In [ ]:
labelsX = ['COUNTRY_NAME', 'INSECTICIDE_CLASS', 'INSECTICIDE_TYPE', 'INSECTICIDE_CONC', 'MOSQUITO_NUMBER','SPECIES']

mosq_csv_ = mosq_csv[labelsX + ['RESISTANCE_STATUS']]
# mosq_csv_Y=mosq_csv['RESISTANCE_STATUS']

print(mosq_csv_.nunique())
# print(mosq_csv_Y.nunique())

COUNTRY_NAME          87
INSECTICIDE_CLASS      6
INSECTICIDE_TYPE      18
INSECTICIDE_CONC      18
MOSQUITO_NUMBER      431
SPECIES               92
RESISTANCE_STATUS      4
dtype: int64


In [ ]:
mosq_csv_ = mosq_csv_.replace(['', 'Not available', 'Undetermined'],np.NaN)

In [ ]:
mosq_csv_.count()

COUNTRY_NAME         21074
INSECTICIDE_CLASS    21074
INSECTICIDE_TYPE     21074
INSECTICIDE_CONC     21074
MOSQUITO_NUMBER      18620
SPECIES              21074
RESISTANCE_STATUS    20355
dtype: int64

In [ ]:
mosq_csv_.isnull().sum()

COUNTRY_NAME            0
INSECTICIDE_CLASS       0
INSECTICIDE_TYPE        0
INSECTICIDE_CONC        0
MOSQUITO_NUMBER      2454
SPECIES                 0
RESISTANCE_STATUS     719
dtype: int64

In [ ]:
mosq_csv_ = mosq_csv_.dropna()
mosq_csv_.dtypes

COUNTRY_NAME         object
INSECTICIDE_CLASS    object
INSECTICIDE_TYPE     object
INSECTICIDE_CONC     object
MOSQUITO_NUMBER      object
SPECIES              object
RESISTANCE_STATUS    object
dtype: object

In [ ]:
mosq_csv_.count()

COUNTRY_NAME         17928
INSECTICIDE_CLASS    17928
INSECTICIDE_TYPE     17928
INSECTICIDE_CONC     17928
MOSQUITO_NUMBER      17928
SPECIES              17928
RESISTANCE_STATUS    17928
dtype: int64

In [ ]:
mosq_csv_.isnull().sum()

COUNTRY_NAME         0
INSECTICIDE_CLASS    0
INSECTICIDE_TYPE     0
INSECTICIDE_CONC     0
MOSQUITO_NUMBER      0
SPECIES              0
RESISTANCE_STATUS    0
dtype: int64

# Creating two datasets: With ranges(took the median) and Without ranges(removed the ranges)

In [ ]:
ranges_mosq = mosq_csv_.copy()

# Filter rows with ~100 and ranges
selected_rows = mosq_csv_[
    mosq_csv_['MOSQUITO_NUMBER'].str.startswith('~') | mosq_csv_['MOSQUITO_NUMBER'].str.contains('-')
]

# Process ~100 values and ranges
selected_rows['MOSQUITO_NUMBER'] = selected_rows['MOSQUITO_NUMBER'].str.replace('~', '')  # Remove ~ from ~100
selected_rows['MOSQUITO_NUMBER'] = selected_rows['MOSQUITO_NUMBER'].str.replace('-', ' ').astype(str)  # Replace '-' with ' '
selected_rows['MOSQUITO_NUMBER'] = selected_rows['MOSQUITO_NUMBER'].str.split().apply(lambda x: sum(map(float, x))/len(x))  # Calculate median for ranges

# Update the original DataFrame with the processed rows
ranges_mosq['MOSQUITO_NUMBER'].update(selected_rows['MOSQUITO_NUMBER'])

<ipython-input-12-abd3d9f74844>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['MOSQUITO_NUMBER'] = selected_rows['MOSQUITO_NUMBER'].str.replace('~', '')  # Remove ~ from ~100
<ipython-input-12-abd3d9f74844>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['MOSQUITO_NUMBER'] = selected_rows['MOSQUITO_NUMBER'].str.replace('-', ' ').astype(str)  # Replace '-' with ' '
<ipython-input-12-abd3d9f74844>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fr

In [ ]:
print("\nDataframe with ranges:")
print(ranges_mosq['MOSQUITO_NUMBER'].tail(5))
print(ranges_mosq.count())


Dataframe with ranges:
21069    100
21070    100
21071     48
21072     40
21073     40
Name: MOSQUITO_NUMBER, dtype: object
COUNTRY_NAME         17928
INSECTICIDE_CLASS    17928
INSECTICIDE_TYPE     17928
INSECTICIDE_CONC     17928
MOSQUITO_NUMBER      17928
SPECIES              17928
RESISTANCE_STATUS    17928
dtype: int64


In [ ]:
# Filter rows without ~100 and ranges
values_mosq = mosq_csv_[~mosq_csv_['MOSQUITO_NUMBER'].str.contains('-')]
# Filter rows ~100
selected_rows = mosq_csv_[mosq_csv_['MOSQUITO_NUMBER'].str.startswith('~') ]

# Process ~100 values and ranges
selected_rows['MOSQUITO_NUMBER'] = selected_rows['MOSQUITO_NUMBER'].str.replace('~', '')

# Update the original DataFrame with the processed rows
values_mosq['MOSQUITO_NUMBER'].update(selected_rows['MOSQUITO_NUMBER'])

<ipython-input-14-6ad2c1a32fbe>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['MOSQUITO_NUMBER'] = selected_rows['MOSQUITO_NUMBER'].str.replace('~', '')
<ipython-input-14-6ad2c1a32fbe>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values_mosq['MOSQUITO_NUMBER'].update(selected_rows['MOSQUITO_NUMBER'])


In [ ]:
print("\nDataframe with values:")
print('~100' in values_mosq['MOSQUITO_NUMBER'].values)
print(values_mosq.count())


Dataframe with values:
False
COUNTRY_NAME         17694
INSECTICIDE_CLASS    17694
INSECTICIDE_TYPE     17694
INSECTICIDE_CONC     17694
MOSQUITO_NUMBER      17694
SPECIES              17694
RESISTANCE_STATUS    17694
dtype: int64


In [ ]:
# Check which rows contain 'µg' in the INSECTICIDE_CONC column
contains_microgram_v = values_mosq['INSECTICIDE_CONC'].str.contains('µg')
contains_microgram_r = ranges_mosq['INSECTICIDE_CONC'].str.contains('µg')

# Invert the boolean mask to select rows that do not contain 'µg'
rows_to_keep_v = ~contains_microgram_v
rows_to_keep_r = ~contains_microgram_r

# Filter the DataFrame to keep only the rows without 'µg' in the INSECTICIDE_CONC column
values_mosq = values_mosq[rows_to_keep_v]
ranges_mosq = ranges_mosq[rows_to_keep_r]

In [ ]:
def convert_percentage_to_number(percentage):
    if pd.isna(percentage):  # Check if the value is NaN
        return None
    else:
        return float(percentage.strip('%')) / 100  # Convert percentage to number

# Apply the function to the 'INSECTICIDE_CONC' column
values_mosq['INSECTICIDE_CONC'] = values_mosq['INSECTICIDE_CONC'].apply(convert_percentage_to_number)
ranges_mosq['INSECTICIDE_CONC'] = ranges_mosq['INSECTICIDE_CONC'].apply(convert_percentage_to_number)

In [ ]:
print(ranges_mosq['INSECTICIDE_CONC'].head(20))

0     0.0010
2     0.0005
4     0.0005
8     0.0005
9     0.0400
11    0.0005
12    0.0010
13    0.0075
14    0.0075
15    0.0075
18    0.0400
20    0.0075
23    0.0400
24    0.0005
26    0.0050
27    0.0400
28    0.0100
29    0.0500
30    0.0400
31    0.0005
Name: INSECTICIDE_CONC, dtype: float64


In [ ]:
print(values_mosq['INSECTICIDE_CONC'].head(20))

0     0.0010
2     0.0005
4     0.0005
8     0.0005
9     0.0400
11    0.0005
12    0.0010
13    0.0075
14    0.0075
15    0.0075
18    0.0400
20    0.0075
23    0.0400
24    0.0005
26    0.0050
27    0.0400
28    0.0100
29    0.0500
30    0.0400
31    0.0005
Name: INSECTICIDE_CONC, dtype: float64


In [ ]:
values_mosq['MOSQUITO_NUMBER'] = values_mosq['MOSQUITO_NUMBER'].astype('float64')
values_mosq['INSECTICIDE_CONC'] = values_mosq['INSECTICIDE_CONC'].astype('float64')

ranges_mosq['MOSQUITO_NUMBER'] = ranges_mosq['MOSQUITO_NUMBER'].astype('float64')
ranges_mosq['INSECTICIDE_CONC'] = ranges_mosq['INSECTICIDE_CONC'].astype('float64')


In [ ]:
values_mosq.dtypes

COUNTRY_NAME          object
INSECTICIDE_CLASS     object
INSECTICIDE_TYPE      object
INSECTICIDE_CONC     float64
MOSQUITO_NUMBER      float64
SPECIES               object
RESISTANCE_STATUS     object
dtype: object

In [ ]:
ranges_mosq.dtypes

COUNTRY_NAME          object
INSECTICIDE_CLASS     object
INSECTICIDE_TYPE      object
INSECTICIDE_CONC     float64
MOSQUITO_NUMBER      float64
SPECIES               object
RESISTANCE_STATUS     object
dtype: object

- Removing MORTALITY_ADJUSTED range values
- Removing NaNs and Not Availables
- Removing ug values from INSECTICIDE_CONC

MORTALITY_ADJUSTED RANGE TYPICALLY
1. 98 to 100 = Susceptible
2. 90 to 98 = Possible Resistance
3. Remaining is Confirmed Resistance

DATASET
- With ranges in MOSQUITO_NUMBER
- Without ranges in MOSQUITO_NUMBER

# Encoding

In [ ]:
def encode_categorical_columns(df, columns):
    encoders = {}
    for column in columns:
        encoder = LabelEncoder()
        encoded_values = encoder.fit_transform(df[column])
        df[column] = encoded_values
        encoders[column] = encoder
    return df, encoders

In [ ]:
columns_to_encode = ['RESISTANCE_STATUS', 'COUNTRY_NAME', 'INSECTICIDE_CLASS', 'INSECTICIDE_TYPE','SPECIES']

values_mosq, encoders_v = encode_categorical_columns(values_mosq.copy(), columns_to_encode)

ranges_mosq, encoders_r = encode_categorical_columns(ranges_mosq.copy(), columns_to_encode)

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter
import pandas as pd

# Calculate the number of samples in class 0
class_0_count = len(ranges_mosq[ranges_mosq['RESISTANCE_STATUS'] == 0])

# Calculate the number of samples in class 2
class_2_count = len(ranges_mosq[ranges_mosq['RESISTANCE_STATUS'] == 2])

# Define the oversampling strategy for class 1
oversample_strategy = {1: class_2_count}

# Define the undersampling strategy for class 0
undersample_strategy = {0: class_2_count}

# Define the SMOTE object
oversampler = SMOTE(sampling_strategy=oversample_strategy)

# Define the RandomUnderSampler object
undersampler = RandomUnderSampler(sampling_strategy=undersample_strategy)

# Define the pipeline
pipeline = Pipeline([
    ('under', undersampler),
    ('over', oversampler),
])

# Apply the pipeline to the dataframe
X_resampled, y_resampled = pipeline.fit_resample(ranges_mosq.drop(columns=['RESISTANCE_STATUS']), ranges_mosq['RESISTANCE_STATUS'])

# Combine X_resampled and y_resampled into a new DataFrame
ranges_mosq = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled, columns=['RESISTANCE_STATUS'])], axis=1)

# Verify the class distribution
print("Class distribution after resampling:")
print(Counter(ranges_mosq['RESISTANCE_STATUS']))

Class distribution after resampling:
Counter({0: 6130, 1: 6130, 2: 6130})


In [ ]:
# Print the mapping of original values to encoded values for each column
for column, encoder in encoders_r.items():
    print(f"Mapping of original values to encoded values for {column}:")
    for original, encoded in zip(encoder.classes_, encoder.transform(encoder.classes_)):
        print(f"{original}: {encoded}")
    print("\n")

Mapping of original values to encoded values for RESISTANCE_STATUS:
Confirmed resistance: 0
Possible resistance: 1
Susceptible: 2


Mapping of original values to encoded values for COUNTRY_NAME:
Afghanistan: 0
Angola: 1
Bangladesh: 2
Benin: 3
Bhutan: 4
Bolivia (Plurinational State of): 5
Botswana: 6
Burkina Faso: 7
Burundi: 8
Cabo Verde: 9
Cambodia: 10
Cameroon: 11
Central African Republic: 12
Chad: 13
China: 14
Colombia: 15
Comoros: 16
Congo: 17
Côte d’Ivoire: 18
Democratic Republic of the Congo: 19
Djibouti: 20
Dominican Republic: 21
Ecuador: 22
Equatorial Guinea: 23
Eritrea: 24
Eswatini: 25
Ethiopia: 26
Gabon: 27
Gambia: 28
Ghana: 29
Guatemala: 30
Guinea: 31
Guinea-Bissau: 32
Haiti: 33
India: 34
Indonesia: 35
Iran (Islamic Republic of): 36
Kenya: 37
Lao People's Democratic Republic: 38
Liberia: 39
Madagascar: 40
Malawi: 41
Malaysia: 42
Mali: 43
Mauritania: 44
Mexico: 45
Morocco: 46
Mozambique: 47
Myanmar: 48
Namibia: 49
Nepal: 50
Nicaragua: 51
Niger: 52
Nigeria: 53
Pakistan: 54
Pana

In [ ]:
# Display the updated DataFrame
print("\nUpdated DataFrame:")
print(ranges_mosq[['RESISTANCE_STATUS', 'COUNTRY_NAME', 'INSECTICIDE_CLASS', 'INSECTICIDE_TYPE','SPECIES']].head(20))


Updated DataFrame:
    RESISTANCE_STATUS  COUNTRY_NAME  INSECTICIDE_CLASS  INSECTICIDE_TYPE  \
0                   0            67                  2                11   
1                   0            11                  3                 6   
2                   0             7                  3                 6   
3                   0             5                  3                 0   
4                   0            68                  3                 6   
5                   0            77                  3                 0   
6                   0            62                  3                 0   
7                   0            26                  3                 8   
8                   0            78                  3                 6   
9                   0            37                  3                 6   
10                  0            14                  2                11   
11                  0            68                  3              

# Seperating Values_mosq & Ranges_mosq into target and predictor dataframes

In [ ]:
labelsX = ['COUNTRY_NAME', 'INSECTICIDE_CLASS', 'INSECTICIDE_TYPE', 'INSECTICIDE_CONC', 'MOSQUITO_NUMBER','SPECIES']

values_mosq_X = values_mosq[labelsX]
values_mosq_Y=values_mosq['RESISTANCE_STATUS']

ranges_mosq_X = ranges_mosq[labelsX]
ranges_mosq_Y=ranges_mosq['RESISTANCE_STATUS']

In [ ]:
print(values_mosq_X.nunique())
print(values_mosq_Y.nunique())

COUNTRY_NAME          81
INSECTICIDE_CLASS      4
INSECTICIDE_TYPE      15
INSECTICIDE_CONC      11
MOSQUITO_NUMBER      405
SPECIES               86
dtype: int64
3


In [ ]:
print(ranges_mosq_X.nunique())
print(ranges_mosq_Y.nunique())

COUNTRY_NAME           81
INSECTICIDE_CLASS       4
INSECTICIDE_TYPE       15
INSECTICIDE_CONC     1456
MOSQUITO_NUMBER      2264
SPECIES                86
dtype: int64
3


In [ ]:
values_mosq_X.describe()

,COUNTRY_NAME,INSECTICIDE_CLASS,INSECTICIDE_TYPE,INSECTICIDE_CONC,MOSQUITO_NUMBER,SPECIES
count,16747.000000,16747.000000,16747.000000,16747.000000,16747.000000,16747.000000
mean,42.182839,2.181943,7.013256,0.012331,98.206556,31.104437
std,24.061307,1.108112,3.687344,0.017737,86.450560,21.565213
min,0.000000,0.000000,0.000000,0.000500,11.000000,0.000000
25%,19.000000,1.000000,5.000000,0.000500,75.000000,14.000000
50%,38.000000,3.000000,6.000000,0.001000,100.000000,29.000000
75%,67.000000,3.000000,10.000000,0.010000,100.000000,30.000000
max,80.000000,3.000000,14.000000,0.050000,2899.000000,85.000000


In [ ]:
values_mosq_Y.describe()

count    16747.000000
mean         0.852033
std          0.921087
min          0.000000
25%          0.000000
50%          0.000000
75%          2.000000
max          2.000000
Name: RESISTANCE_STATUS, dtype: float64

In [ ]:
ranges_mosq_X.describe()

,COUNTRY_NAME,INSECTICIDE_CLASS,INSECTICIDE_TYPE,INSECTICIDE_CONC,MOSQUITO_NUMBER,SPECIES
count,18390.00000,18390.000000,18390.000000,18390.000000,18390.000000,18390.000000
mean,42.92311,2.153453,6.954051,0.012141,97.766510,31.229744
std,23.95033,1.107053,3.650452,0.017379,86.616005,21.761863
min,0.00000,0.000000,0.000000,0.000500,11.000000,0.000000
25%,26.00000,1.000000,5.000000,0.000500,74.000000,14.000000
50%,40.00000,3.000000,6.000000,0.001000,100.000000,29.000000
75%,67.00000,3.000000,10.000000,0.010000,100.000000,30.000000
max,80.00000,3.000000,14.000000,0.050000,2899.000000,85.000000


In [ ]:
ranges_mosq_Y.describe()

count    18390.000000
mean         1.000000
std          0.816519
min          0.000000
25%          0.000000
50%          1.000000
75%          2.000000
max          2.000000
Name: RESISTANCE_STATUS, dtype: float64

# Scaling

In [ ]:
# Select the columns to be normalized
columns_to_normalize = ['INSECTICIDE_CONC', 'MOSQUITO_NUMBER']

# Initialize the MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the selected columns
values_mosq_X[columns_to_normalize] = scaler.fit_transform(values_mosq_X[columns_to_normalize])
ranges_mosq_X[columns_to_normalize] = scaler.fit_transform(ranges_mosq_X[columns_to_normalize])


<ipython-input-35-9cf635af68eb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  values_mosq_X[columns_to_normalize] = scaler.fit_transform(values_mosq_X[columns_to_normalize])
<ipython-input-35-9cf635af68eb>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ranges_mosq_X[columns_to_normalize] = scaler.fit_transform(ranges_mosq_X[columns_to_normalize])


In [ ]:
values_mosq_X.describe()

,COUNTRY_NAME,INSECTICIDE_CLASS,INSECTICIDE_TYPE,INSECTICIDE_CONC,MOSQUITO_NUMBER,SPECIES
count,16747.000000,16747.000000,16747.000000,16747.000000,16747.000000,16747.000000
mean,42.182839,2.181943,7.013256,0.239017,0.030196,31.104437
std,24.061307,1.108112,3.687344,0.358331,0.029934,21.565213
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,19.000000,1.000000,5.000000,0.000000,0.022161,14.000000
50%,38.000000,3.000000,6.000000,0.010101,0.030817,29.000000
75%,67.000000,3.000000,10.000000,0.191919,0.030817,30.000000
max,80.000000,3.000000,14.000000,1.000000,1.000000,85.000000


In [ ]:
ranges_mosq_X.describe()

,COUNTRY_NAME,INSECTICIDE_CLASS,INSECTICIDE_TYPE,INSECTICIDE_CONC,MOSQUITO_NUMBER,SPECIES
count,18390.00000,18390.000000,18390.000000,18390.000000,18390.000000,18390.000000
mean,42.92311,2.153453,6.954051,0.235179,0.030044,31.229744
std,23.95033,1.107053,3.650452,0.351100,0.029992,21.761863
min,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,26.00000,1.000000,5.000000,0.000000,0.021814,14.000000
50%,40.00000,3.000000,6.000000,0.010101,0.030817,29.000000
75%,67.00000,3.000000,10.000000,0.191919,0.030817,30.000000
max,80.00000,3.000000,14.000000,1.000000,1.000000,85.000000


# Training values_csv

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(values_mosq_X, values_mosq_Y, test_size=0.30, random_state=42)

In [ ]:
# KNN
knn_params = {'n_neighbors': [3, 5, 11, 19], 'weights': ['uniform', 'distance']}
knn_grid = GridSearchCV(KNeighborsClassifier(), knn_params, cv=5)
knn_grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 11, 19],
                         'weights': ['uniform', 'distance']})

In [ ]:
dt_params = {'max_depth': [4, 5, 6, 7, 8, 9, 10, 15, None], 'min_samples_leaf': [1, 2, 3, 4, 5], 'min_samples_split': list(range(2,10))}
dt_grid = GridSearchCV(DecisionTreeClassifier(), dt_params, cv=2, verbose = 10, n_jobs = -1)
dt_grid.fit(X_train, y_train)

Fitting 2 folds for each of 360 candidates, totalling 720 fits


GridSearchCV(cv=2, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [4, 5, 6, 7, 8, 9, 10, 15, None],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9]},
             verbose=10)

In [ ]:
# Naive Bayes
nb_model = GaussianNB()
nb_scores = cross_val_score(nb_model, X_train, y_train, cv=5)

In [ ]:
# SVM with linear kernel
svm_linear_params = {'C': [0.1, 1, 10]}
svm_linear_grid = GridSearchCV(SVC(kernel='linear'), svm_linear_params, cv=5)
svm_linear_grid.fit(X_train, y_train)

In [ ]:
# SVM with non-linear kernel (RBF)
svm_rbf_params = {'C': [0.1, 1, 10], 'gamma': [1, 0.1, 0.01]}
svm_rbf_grid = GridSearchCV(SVC(kernel='rbf'), svm_rbf_params, cv=5)
svm_rbf_grid.fit(X_train, y_train)

# Model Evaluation for values_csv


In [ ]:
# Function to evaluate the model
def evaluate_multiclass_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test) if hasattr(model, 'predict_proba') else model.decision_function(X_test)
    n_classes = y_test.nunique()

    # Compute one-hot encoding for y_test and y_pred
    y_test_one_hot = label_binarize(y_test, classes=list(range(n_classes)))
    y_pred_one_hot = label_binarize(y_pred, classes=list(range(n_classes)))

    # Compute multiclass AUC-ROC
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    for i in range(n_classes):
        fpr[i], tpr[i], _ = roc_curve(y_test_one_hot[:, i], y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    # Compute macro-average ROC curve and AUC
    fpr_macro = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))
    tpr_macro = np.zeros_like(fpr_macro)
    for i in range(n_classes):
        tpr_macro += np.interp(fpr_macro, fpr[i], tpr[i])
    tpr_macro /= n_classes
    roc_auc_macro = auc(fpr_macro, tpr_macro)

    # Compute accuracy, F1-score, precision, recall, and confusion matrix
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    conf_matrix = confusion_matrix(y_test, y_pred)

    return {
        'Accuracy': accuracy,
        'F1 Score': f1,
        'Precision': precision,
        'Recall': recall,
        'AUC ROC (Macro)': roc_auc_macro,
        'AUC ROC (Per Class)': roc_auc,
        'ROC Curve (Macro)': (fpr_macro, tpr_macro),
        'Confusion Matrix': conf_matrix
    }


In [ ]:
# Evaluating each model
dt_evaluation_v = evaluate_multiclass_model(dt_grid.best_estimator_, X_test, y_test)
knn_evaluation_v = evaluate_multiclass_model(knn_grid.best_estimator_, X_test, y_test)
# nb_evaluation_v = evaluate_multiclass_model(nb_model, X_test, y_test) # Naive Bayes does not use predict_proba or decision_function
# # svm_linear_evaluation_v = evaluate_multiclass_model(svm_linear_grid.best_estimator_, X_test, y_test)
# # svm_rbf_evaluation_v = evaluate_multiclass_model(svm_rbf_grid.best_estimator_, X_test, y_test)

In [ ]:
dt_best_params_v = dt_grid.best_params_
print(f"Decision Tree best params: {dt_best_params_v}")
print("Decision Tree Evaluation:", dt_evaluation_v)

Decision Tree best params: {'max_depth': 15, 'min_samples_leaf': 3, 'min_samples_split': 9}
Decision Tree Evaluation: {'Accuracy': 0.7176119402985075, 'F1 Score': 0.5759302213670662, 'Precision': 0.5947013824040713, 'Recall': 0.575567643546261, 'AUC ROC (Macro)': 0.7908096382004066, 'AUC ROC (Per Class)': {0: 0.8539369071228655, 1: 0.6570046454899288, 2: 0.8614689507951829}, 'ROC Curve (Macro)': (array([0.        , 0.00296128, 0.00387244, 0.00432802, 0.00865604,
       0.00933941, 0.01708428, 0.0191344 , 0.01936219, 0.02277904,
       0.02358052, 0.02420105, 0.02451132, 0.0257524 , 0.02699348,
       0.02870159, 0.0288551 , 0.03052392, 0.03071672, 0.0309795 ,
       0.0319578 , 0.03381942, 0.03446861, 0.03487895, 0.03528929,
       0.03569963, 0.03692212, 0.03723239, 0.03734099, 0.03857201,
       0.03909401, 0.0393927 , 0.03980304, 0.04021338, 0.04188644,
       0.04219671, 0.04267542, 0.04529941, 0.0455478 , 0.04623022,
       0.04718917, 0.04759951, 0.04829157, 0.04924087, 0.0498861

In [ ]:
knn_best_params_v = knn_grid.best_params_
# svm_linear_best_params_v = svm_linear_grid.best_params_
# svm_rbf_best_params_v = svm_rbf_grid.best_params_

In [ ]:
print(f"KNN best params: {knn_best_params_v}")
# print(f"SVM Linear best params: {svm_linear_best_params_v}")
# print(f"SVM RBF best params: {svm_rbf_best_params_v}")

KNN best params: {'n_neighbors': 19, 'weights': 'distance'}


In [ ]:
print("KNN Evaluation:", knn_evaluation_v)
# print("Naive Bayes Evaluation:", nb_evaluation_v)
# print("SVM Linear Evaluation:", svm_linear_evaluation_v)
# print("SVM RBF Evaluation:", svm_rbf_evaluation_v)

KNN Evaluation: {'Accuracy': 0.7243781094527363, 'F1 Score': 0.5908519030852667, 'Precision': 0.6088161678783366, 'Recall': 0.5879790112242931, 'AUC ROC (Macro)': 0.7946642807714469, 'AUC ROC (Per Class)': {0: 0.8553196660956568, 1: 0.66601079762524, 2: 0.8624608848099623}, 'ROC Curve (Macro)': (array([0.        , 0.01275626, 0.01298405, ..., 0.65695527, 0.65736561,
       1.        ]), array([0.        , 0.09631676, 0.09815856, ..., 0.91083957, 0.91092973,
       1.        ])), 'Confusion Matrix': array([[2209,  120,  259],
       [ 324,  114,  197],
       [ 364,  121, 1317]])}


# Training ranges_csv


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(ranges_mosq_X, ranges_mosq_Y, test_size=0.30, random_state=42)

In [ ]:
dt_params = {'max_depth': [4, 5, 6, 7, 8, 9, 10, 15, None], 'min_samples_leaf': [1, 2, 3, 4, 5], 'min_samples_split': list(range(2,10))}
dt_grid = GridSearchCV(DecisionTreeClassifier(), dt_params, cv=2, verbose = 10, n_jobs = -1)
dt_grid.fit(X_train, y_train)

Fitting 2 folds for each of 360 candidates, totalling 720 fits


GridSearchCV(cv=2, estimator=DecisionTreeClassifier(), n_jobs=-1,
             param_grid={'max_depth': [4, 5, 6, 7, 8, 9, 10, 15, None],
                         'min_samples_leaf': [1, 2, 3, 4, 5],
                         'min_samples_split': [2, 3, 4, 5, 6, 7, 8, 9]},
             verbose=10)

# Model evaluation for ranges_csv

In [ ]:
# Evaluating each model
dt_evaluation_r = evaluate_multiclass_model(dt_grid.best_estimator_, X_test, y_test)

In [ ]:
dt_best_params_r = dt_grid.best_params_
print(f"Decision Tree best params: {dt_best_params_r}")
print("Decision Tree Evaluation:", dt_evaluation_r)

Decision Tree best params: {'max_depth': 10, 'min_samples_leaf': 5, 'min_samples_split': 3}
Decision Tree Evaluation: {'Accuracy': 0.7211084905660378, 'F1 Score': 0.5542390884989717, 'Precision': 0.5945089781287177, 'Recall': 0.5621542688260187, 'AUC ROC (Macro)': 0.7863785628231537, 'AUC ROC (Per Class)': {0: 0.8474403757059373, 1: 0.6582100229086081, 2: 0.853468700821822}, 'ROC Curve (Macro)': (array([0.00000000e+00, 2.25073149e-04, 9.00292595e-04, 1.57551204e-03,
       2.70087779e-03, 3.37609723e-03, 4.50146298e-03, 5.40175557e-03,
       6.07697502e-03, 6.75219446e-03, 7.87756021e-03, 9.45307225e-03,
       9.73630832e-03, 1.17647059e-02, 1.21703854e-02, 1.25458996e-02,
       1.29817444e-02, 1.31578947e-02, 1.40758874e-02, 1.46297547e-02,
       1.56058752e-02, 1.68298654e-02, 1.68804862e-02, 1.70385396e-02,
       1.71358629e-02, 1.80538556e-02, 1.83598531e-02, 1.86658507e-02,
       1.86810713e-02, 2.05018360e-02, 2.14198286e-02, 2.23378213e-02,
       2.27323880e-02, 2.3935091